In [2]:
import zipfile
f=zipfile.ZipFile("/content/test.csv.zip")
for file in f.namelist():
  f.extract(file,"/content/")
f.close()

In [30]:
import pandas as pd

In [31]:
df=pd.read_csv('/content/train.csv')

In [32]:
df.head(10)

,Unnamed: 0,review,label
0,0,I think they really let the quality of the DVD...,0
1,1,I'm sorry but this is just awful. I have told ...,0
2,2,"The Japenese sense of pacing, editing and musi...",0
3,3,"In the '60's/'70's, David Jason was renowned f...",1
4,4,"""Hail The Woman"" is one of the most moving fil...",1
5,5,"Budget limitations, time restrictions, shootin...",1
6,6,"""Jaded"" should not be considered as en erotic ...",1
7,7,I give it 8 out of 10 because it is a cult cla...,1
8,8,"With ""Batman Returns"", Tim Burton succumbed to...",1
9,9,Saw this at the video store and thought I'd gi...,0


In [33]:
df['review'][1000]

'Comedy works best when it relates to stuff that\'s true. But even as such, some effort is required to make jokes that everyone likes and even the most grumpy of viewers can crack a smile. When I look at the Daily Show, I see the whole "it\'s funny because it\'s true" thing, but I don\'t, however, see the effort and often times I don\'t if they\'re being funny or just trying to make a point(I notice this mostly in the interview segments). The Daily Show started off as a news parody, by definition they poke fun at how the media plays it\'s own news by pretending to be inept and dumb news reporters and anchormen and they tackled tons of subjects from science to movies and sometimes politics, then Jon Stewart came along...and it all went to Hell. Thr first years of Jon Stewart\'s reign were arguably golden, I though he was so funny, but then 2004 came along and it\'s where you start to notice a huge chance in the show from there on. The show\'s humor has gone stale, Colbert left, Steve Ca

In [34]:
pip install tensorflow

In [35]:
import numpy as np
import pandas as pd
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dropout, Dense, Bidirectional, Attention, Input, Concatenate
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import LSTM
from tensorflow.keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.regularizers import l2
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping

In [36]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import string
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('punkt_tab')



stop_words = set(stopwords.words('english'))
def preprocess(text):
    text = text.lower()
    text = ''.join([word for word in text if word not in string.punctuation])
    tokens = word_tokenize(text)
    tokens = [word for word in tokens if word not in stop_words]
    return ' '.join(tokens)


df['cleaned_review'] = df['review'].apply(preprocess)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


In [37]:
print(df.columns)

Index(['Unnamed: 0', 'review', 'label', 'cleaned_review'], dtype='object')


In [38]:
X_train, X_test, y_train, y_test = train_test_split(df['cleaned_review'], df['label'], test_size=0.2, random_state=42)

In [39]:
max_words = 15000 # 词汇表大小
max_len = 250 # 每个序列的最大长度

tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(X_train)

X_train_seq = tokenizer.texts_to_sequences(X_train)
X_test_seq = tokenizer.texts_to_sequences(X_test)

X_train_pad = pad_sequences(X_train_seq, maxlen=max_len)
X_test_pad = pad_sequences(X_test_seq, maxlen=max_len)

In [40]:
model = Sequential()
model.add(Embedding(input_dim=max_words, output_dim=128, input_length=max_len))
model.add(LSTM(128, return_sequences=True))
model.add(Dropout(0.5))
model.add(LSTM(64))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid', kernel_regularizer=l2(0.01)))

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [61]:
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=2, min_lr=1e-5)

history = model.fit(
    X_train_pad,
    y_train,
    epochs=10,
    batch_size=32,
    validation_split=0.2,
    callbacks=[reduce_lr]
)

Epoch 1/10
800/800 ━━━━━━━━━━━━━━━━━━━━ 24s 30ms/step - accuracy: 0.9896 - loss: 0.0493 - val_accuracy: 0.8827 - val_loss: 0.4610 - learning_rate: 2.0000e-04
Epoch 2/10
800/800 ━━━━━━━━━━━━━━━━━━━━ 18s 22ms/step - accuracy: 0.9964 - loss: 0.0308 - val_accuracy: 0.8766 - val_loss: 0.4975 - learning_rate: 2.0000e-04
Epoch 3/10
800/800 ━━━━━━━━━━━━━━━━━━━━ 18s 22ms/step - accuracy: 0.9972 - loss: 0.0255 - val_accuracy: 0.8781 - val_loss: 0.5216 - learning_rate: 2.0000e-04
Epoch 4/10
800/800 ━━━━━━━━━━━━━━━━━━━━ 19s 23ms/step - accuracy: 0.9980 - loss: 0.0223 - val_accuracy: 0.8766 - val_loss: 0.5294 - learning_rate: 4.0000e-05
Epoch 5/10
800/800 ━━━━━━━━━━━━━━━━━━━━ 19s 22ms/step - accuracy: 0.9981 - loss: 0.0222 - val_accuracy: 0.8777 - val_loss: 0.5414 - learning_rate: 4.0000e-05
Epoch 6/10
800/800 ━━━━━━━━━━━━━━━━━━━━ 17s 21ms/step - accuracy: 0.9983 - loss: 0.0218 - val_accuracy: 0.8780 - val_loss: 0.5437 - learning_rate: 1.0000e-05
Epoch 7/10
800/800 ━━━━━━━━━━━━━━━━━━━━ 17s 21ms/ste

In [62]:
df_test=pd.read_csv('/content/test.csv')
df_test['review'] = df_test['review'].apply(preprocess)

In [63]:
df_test['review']

,review
0,make orbreak love story whether like character...
1,bend like beckham film got little exposure uni...
2,petes meteor seen referred authentic gem caugh...
3,funny find forced review movie ambr br reviewi...
4,bare wench another softcore parody blair witch...
...,...
9995,reason wanted see orlando bloom simply put mov...
9996,three russian aristocrats soak decadence monte...
9997,greetings darkness remember happened woody all...
9998,film tour de force julie taymor directs stage ...


In [64]:
max_words = 15000 # 词汇表大小
max_len = 250 # 每个序列的最大长度
tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(df_test['review'])

y_seq = tokenizer.texts_to_sequences(df_test['review'])

y_pad = pad_sequences(y_seq, maxlen=max_len)

In [65]:
y_pred_prob=model.predict(y_pad)
y_pred = (y_pred_prob > 0.5).astype(int)

313/313 ━━━━━━━━━━━━━━━━━━━━ 2s 8ms/step


In [66]:
df_test.insert(0, 'id', range(0, len(df_test)))

In [68]:
df_test.drop(columns=['Unnamed: 0'],inplace=True)
df_test.drop(columns=['review'],inplace=True)

In [69]:
df_test

,id
0,0
1,1
2,2
3,3
4,4
...,...
9995,9995
9996,9996
9997,9997
9998,9998


In [70]:
df_test['Predicted']=y_pred

In [71]:
df_test

,id,Predicted
0,0,0
1,1,1
2,2,0
3,3,0
4,4,0
...,...,...
9995,9995,0
9996,9996,0
9997,9997,0
9998,9998,1


In [72]:
df_test.to_csv('predictions_custom2.csv',index=False)

qqqqqq